In [ ]:
########################################################################################################################
## -- libraries and packages -- ########################################################################################
########################################################################################################################
import os
import sys
sys.path.append(os.path.abspath(".."))
import torch
import transformer

########################################################################################################################
## -- testing the data handler module -- ###############################################################################
########################################################################################################################
src_vocab_path = "../data/vocabs/en_vocab.json"
tgt_vocab_path = "../data/vocabs/fa_vocab.json"
src_path, src_name = "../data/dataset/Tatoeba.zip", "en.txt"
tgt_path, tgt_name = "../data/dataset/Tatoeba.zip", "fa.txt"
SOS_TOKEN, PAD_TOKEN, EOS_TOKEN = '<SOS>', '<PAD>', '<EOS>'

data_handler = transformer.DataHandler(src_path, src_name, src_vocab_path, tgt_path, tgt_name, tgt_vocab_path, 
                                       SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, max_sequence_length = 256, max_sentences = 1000)

data = data_handler.data()

########################################################################################################################
## -- testing the transformer -- #######################################################################################
########################################################################################################################
batch_size, max_sequence_length, model_emb, hidden = 32, 256, 512, 2048
num_heads, dropout_p, num_layers_enc, num_layers_dec = 8, 0.1, 2, 2

transformer_pipeline = transformer.Transformer(model_emb, hidden, num_heads, dropout_p, num_layers_enc, 
                                               num_layers_dec, max_sequence_length, data.src_stoi, data.tgt_stoi, 
                                               SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, device = 'cpu')

en_batch, fa_batch = ("Hi",), ("سلام",)
enc_mask, dec_mask, dec_cross_mask = transformer.MaskGenerator(max_sequence_length = max_sequence_length).generate_masks(en_batch, fa_batch)
transformer_pipeline(en_batch, fa_batch, enc_sos_token = False, enc_eof_token = False, 
                     dec_sos_token = False, dec_eos_token = False, enc_mask = enc_mask, 
                     dec_mask = dec_mask, dec_cross_mask = dec_cross_mask).shape

torch.Size([1, 256, 85])